# **Team name: UINSUSKA-Mandiri**

Task 1A and 1B : Hate Speech classification in Sinhala and Gujarati

In [ ]:
# read data from folder

import numpy as np
import os
import pandas as pd
from tqdm.auto import tqdm
from gensim.models import FastText
from nltk.tokenize import word_tokenize
import re

path_data = 'hasoc2023-uin/'      # adjust the path to your data sources
train_setA = "train_sinhala.csv"  # file name was changed to our standard convention name:
test_setA = "test_sinhala.csv"    # train_<language>.csv and test_<language>.csv
train_setB = "train_gujarati.csv"
test_setB = "test_gujarati.csv"

df_train = pd.read_csv(path_data+train_setA, sep='\t')   #check the separator of csv file,
df_test = pd.read_csv(path_data+test_setA, sep=',')      #there are differences among all files
df_train

,post_id,text,label
0,726758237668659201,@USER @USER පට්ට පට පට...,NOT
1,915618589855617026,පරණ කෑල්ල අද වෙනකම් හිටියනම් අදට අවුරුදු 4යි. ...,HOF
2,925001070430040065,යාළුවා කියලා හිතන් සර් ගේ ඔලුවට රෙද්ද දාලා නෙල...,NOT
3,1397219745707986955,හොඳ මිතුරියක් කතා කලා. විස්තර කතාකරමින් ඉදලා ම...,HOF
4,950376113150222336,"ඔය බනින්නෙ.. හරකා, මී හරකා කිය කිය...",HOF
...,...,...,...
7495,930270216612872193,අද උදේ දැක්කා පට්ට ලස්සන හීනයක්,NOT
7496,1159471424613969921,@USER කසල වෙන් කරලා දෙන්න පුරුදු වුනොත් වැඩේ ...,NOT
7497,1073271775583100928,ඒත් පබාගේ පස්ස නම්... ඉස්සෝ කොටුවක් උනත් දාන්න...,HOF
7498,1131018244733657088,සුදුවෑන්වලට මාරම මාර බයක් තියෙන ඈයෝ අන්තවාදයට...,HOF


In [ ]:
# rename column, display data-test

df_test = df_test.set_axis(['tweet_id', 'tweet'], axis='columns', copy=False)
df_test

,tweet_id,tweet
0,task1a_test_1,ඔව් ඔව්.. තොපි ආදරේට කලොත් කරන්නෙ ඔහොම වැඩක් ත...
1,task1a_test_2,ඇයි උඹ අලි පු*න්ද? #SriLankanThreats
2,task1a_test_3,@USER @USER පක ටෝක් දෙන්න එපා
3,task1a_test_4,"මම නවය වසරේදී,පංති නායකයා කාලේ පංතියේ කොස්ස නැ..."
4,task1a_test_5,මේ ආන්ඩුවටනම් හෙන 7ම ගහන්න ඔනේ මුන්ට පේන්නෙ නැ...
...,...,...
2495,task1a_test_2496,රටක ආර්ථිකය කෙලෙසපු මොළකරු පට්ට හොරෙක් ! #SriL...
2496,task1a_test_2497,@USER හරකා මොකා උනත් අවසානෙ මසට යාලු
2497,task1a_test_2498,@USER තොපි වගේ කාළකන්නි මාධ්‍ය ආයතන තහනම් කරන ...
2498,task1a_test_2499,රියැදුරු බලපත්‍ර නිකුත් කිරීම ඇතුළු සියලු කටයු...


In [ ]:
# display data-train

df_train['label'].value_counts(normalize=False)


NOT    4324
HOF    3176
Name: label, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

df_train = df_train.set_axis(['tweet_id', 'tweet', 'label'], axis='columns', copy=False)   #standardize column name
df_train

,tweet_id,tweet,label
0,726758237668659201,@USER @USER පට්ට පට පට...,NOT
1,915618589855617026,පරණ කෑල්ල අද වෙනකම් හිටියනම් අදට අවුරුදු 4යි. ...,HOF
2,925001070430040065,යාළුවා කියලා හිතන් සර් ගේ ඔලුවට රෙද්ද දාලා නෙල...,NOT
3,1397219745707986955,හොඳ මිතුරියක් කතා කලා. විස්තර කතාකරමින් ඉදලා ම...,HOF
4,950376113150222336,"ඔය බනින්නෙ.. හරකා, මී හරකා කිය කිය...",HOF
...,...,...,...
7495,930270216612872193,අද උදේ දැක්කා පට්ට ලස්සන හීනයක්,NOT
7496,1159471424613969921,@USER කසල වෙන් කරලා දෙන්න පුරුදු වුනොත් වැඩේ ...,NOT
7497,1073271775583100928,ඒත් පබාගේ පස්ස නම්... ඉස්සෝ කොටුවක් උනත් දාන්න...,HOF
7498,1131018244733657088,සුදුවෑන්වලට මාරම මාර බයක් තියෙන ඈයෝ අන්තවාදයට...,HOF


In [ ]:
# creating data-dev for validation

shuffled_train = df_train.sample(frac=1, random_state=23).reset_index()
df_train1, df_dev1 = train_test_split(shuffled_train, test_size=0.1)
df_train1

,index,tweet_id,tweet,label
6626,7327,1276189330923335681,"අපි පොඩි කාලෙ අහපු ""ඇම්ඩගෙ කතා"" ඇත්තද එතකොට.? ...",NOT
1357,1790,778300947567026176,"@USER @USER ටහුඩු, හැපි බර්ත්ඩේ ටෝකා පුතේ. සො...",NOT
819,6050,1481279802829320198,@USER ඩ්‍රාමා එච්චර අමාරු ද,NOT
6217,502,618419188156137472,හැමෝම තමුන්ගේ පාසලේ හිටපු පට්ට ඩෑල් ගැන වර්ණා ...,NOT
1105,2974,1186283920842584064,"@USER පහුගිය අවුරැදු හතර හමාරේ වගේ ,",NOT
...,...,...,...,...
5958,1477,627511530980966400,මේ Max Dura මගුලක් නිසා හෙන වදේ බන්. ඇයි අර මු...,NOT
3271,5403,1383455672403054594,අමුතුවෙන් හිතන්නත් දෙයක් මෙතන නැහැ... මේක තමයි...,HOF
6305,4779,693998649282576384,තම දියණියට ලිපියක් දුන් සිසුවා පැහැරගත් පියා ස...,NOT
2344,275,798790313066102784,"@USER හම්බන්තොටයි, මත්තලයි ගැන කතා කරන්න පුලුව...",NOT


**List of Stopword**

In [ ]:
# Stopwords in Sinhala
# Stopwords only used in balancing the data_train
# For Gujarati we don't remove stopwords

stp=['සහ',
'සමග',
'සමඟ',
'අහා',
'ආහ්',
'ආ',
'ඕහෝ',
'අනේ',
'අඳෝ',
'අපොයි',
'අපෝ',
'අයියෝ',
'ආයි',
'ඌයි',
'චී',
'චිහ්',
'චික්',
'හෝ‍',
'දෝ',
'දෝහෝ',
'මෙන්',
'සේ',
'වැනි',
'බඳු',
'වන්',
'අයුරු',
'අයුරින්',
'ලෙස',
'වැඩි',
'ශ්‍රී',
'හා',
'ය',
'නිසා',
'නිසාවෙන්',
'බවට',
'බව',
'බවෙන්',
'නම්',
'වැඩි',
'සිට',
'දී',
'මහා',
'මහ',
'පමණ',
'පමණින්',
'පමන',
'වන',
'විට',
'විටින්',
'මේ',
'මෙලෙස',
'මෙයින්',
'ඇති',
'ලෙස',
'සිදු',
'වශයෙන්',
'යන',
'සඳහා',
'මගින්',
'හෝ‍',
'ඉතා',
'ඒ',
'එම',
'ද',
'අතර',
'විසින්',
'සමග',
'පිළිබඳව',
'පිළිබඳ',
'තුළ',
'බව',
'වැනි',
'මහ',
'මෙම',
'මෙහි',
'මේ',
'වෙත',
'වෙතින්',
'වෙතට',
'වෙනුවෙන්',
'වෙනුවට',
'වෙන',
'ගැන',
'නෑ',
'අනුව',
'නව',
'පිළිබඳ',
'විශේෂ',
'දැනට',
'එහෙන්',
'මෙහෙන්',
'එහේ',
'මෙහේ',
'ම',
'තවත්',
'තව ',
'සහ',
'දක්වා',
'ට',
'ගේ',
'එ',
'ක',
'ක්',
'බවත්',
'බවද',
'මත',
'ඇතුලු',
'ඇතුළු',
'මෙසේ',
'වඩා',
'වඩාත්ම',
'නිති',
'නිතිත්',
'නිතොර',
'නිතර',
'ඉක්බිති',
'දැන්',
'යලි',
'පුන',
'ඉතින්',
'සිට',
'සිටන්',
'පටන්',
'තෙක්',
'දක්වා',
'සා',
'තාක්',
'තුවක්',
'පවා',
'ද',
'හෝ‍',
'වත්',
'විනා',
'හැර',
'මිස',
'මුත්',
'කිම',
'කිම්',
'ඇයි',
'මන්ද',
'හෙවත්',
'නොහොත්',
'පතා',
'පාසා',
'ගානෙ',
'තව',
'ඉතා',
'බොහෝ',
'වහා',
'සෙද',
'සැනින්',
'හනික',
'එම්බා',
'එම්බල',
'බොල',
'නම්',
'වනාහි',
'කලී',
'ඉඳුරා',
'අන්න',
'ඔන්න',
'මෙන්න',
'උදෙසා',
'පිණිස',
'සඳහා',
'අරබයා',
'නිසා',
'එනිසා',
'එබැවින්',
'බැවින්',
'හෙයින්',
'සේක්',
'සේක',
'ගැන',
'අනුව',
'පරිදි',
'විට',
'තෙක්',
'මෙතෙක්',
'මේතාක්',
'තුරු',
'තුරා',
'තුරාවට',
'තුලින්',
'නමුත්',
'එනමුත්',
'වස්',
'මෙන්',
'ලෙස',
'පරිදි',
'එහෙත්',
]

# **BALANCING DATASET**

In [ ]:


#Step 1 data train
dnot = df_train1[df_train1.label == 'NOT']
dhof = df_train1[df_train1.label == 'HOF']
dhof


,index,tweet_id,tweet,label
5197,1132,1285483222550237184,@USER තමුසෙ NSBM ගිහින් ඩිග්‍රියක් කරනවා කියල...,HOF
1508,2519,743994404269096960,අඩෝ පපුව පිච්චුනා වගේ. අැස් දෙකට කදුලු ඉනුවා ඔ...,HOF
3750,4936,1442016935642902528,@USER @USER තෝ පලකෝ සයිබීරියන් කෑහුට්ටී,HOF
2805,4859,1197761471851245573,ඉස්සෙල්ලාම දුන්න පෙම්හසුනට ලැබුන ප්‍රතිචාරය තම...,HOF
4174,2816,1249750123548213248,ටීවි එකේ නිව්ස් වල යනවා දැක්කා. Police officer...,HOF
...,...,...,...,...
5969,7240,1101352281155559427,බලහල්ල රජයේ කැම්පස් වලට බැනපු උන් මොනවගෙද ඉන්...,HOF
3164,2624,1068011964788559872,@USER @USER කොහෙන් ගියත් මූ යන්නෙ ගල විදපු තැ...,HOF
5657,3910,981194683228471297,@USER පිස්සා කිව්වට පිස්සු පාටක් නම් පේන්නෙම නැ..,HOF
2013,2029,1250450251317522432,මොනවද තිරිසනෝ මේ දෙරණේ පෙන්වන ටෙලි නාට්‍ය? පා...,HOF


In [ ]:
# step 2: oversampling the HOF class

# select tweets which have more than 18 words, not include stop words
new_HOF= []

i = 0
for txt in dhof['tweet'] :
    a= txt.split(' ')
    if len(a) > 18:
        baru = [x for x in a if x not in stp]
        new_HOF.append([i+7500,' '.join(baru),'HOF'])
        i += 1
#new_HOF


In [ ]:
# convert into pandas dataframe

df_HOF_additional = pd.DataFrame(np.array(new_HOF),
             columns=['index', 'tweet', 'label'
                      ])
df_HOF_additional

,index,tweet,label
0,7500,@USER තමුසෙ NSBM ගිහින් ඩිග්‍රියක් කරනවා කියල...,HOF
1,7501,අඩෝ පපුව පිච්චුනා වගේ. අැස් දෙකට කදුලු ඉනුවා ඔ...,HOF
2,7502,ඉදගන්න යනකොට ඒකි මොනාදෝ කියනවලු.කඩ්ඩ බැරිඋනත් ...,HOF
3,7503,ලබ්බ බහින සංගමය කියලා වගේ එකක් හැදුවා හරි. Ami...,HOF
4,7504,@USER @USER බෝතල් ගොඩක් කුඩුකරලා දැනුත් පේමන්...,HOF
...,...,...,...
1469,8969,මං තමා එච්චර ගාණක් ෆන්ඩ් කරන්නෙ...නාමල් අයියා ...,HOF
1470,8970,@USER @USER සවේන්ද්‍රයි සයි හින්දා තෝ අද ...,HOF
1471,8971,බලහල්ල රජයේ කැම්පස් වලට බැනපු උන් මොනවගෙද ඉන්...,HOF
1472,8972,@USER @USER කොහෙන් ගියත් මූ යන්නෙ ගල විදපු තැ...,HOF


In [ ]:
# get some tweets with class HOF to be added to data-train (choose by index)
# no need for Gujarati

adds = df_HOF_additional[200:1236]

df1 = pd.concat([dnot,adds,dhof])# .reset_index()
df1['label'].value_counts()


NOT    3893
HOF    3893
Name: label, dtype: int64

In [ ]:
# This is balanced training data, use for SVM Training

shuffled_train = df1.sample(frac=1, random_state=23)
shuffled_train


,index,tweet_id,tweet,label
3030,7099,1.145568e+18,"@USER , @USER @USER ටහුක් තෝ ඉන්නවා ද",HOF
3642,1331,6.756934e+17,ඉස්සර උබ වලියක් නම් ලොවෙත් කියනො ගිහාන් අයියා ...,HOF
539,4881,1.396944e+18,නින්ද යන්නේ නැති එක නම් කරුම ලෙඩක්,NOT
1089,8589,NaN,10 අළු වලින් ගොයම් ගස් පෝෂණය වෙලා හැදීල එන පි...,HOF
196,1169,1.161323e+18,@USER රිචඩ් ද සොයිසා ප්‍රේමකීර්ති ද අල්විස් රෝ...,NOT
...,...,...,...,...
1237,2408,1.185154e+18,@USER නරකද වී නිෂ්පාදන වියදම් අඩුකරලා ගොවියායි...,NOT
5752,668,1.053322e+18,කහපාට රෙද්දකින් තඩි පීප්පයක් ඔතා තිබූ ඒ අවස්තාව,NOT
3387,634,1.166635e+18,ගත මදක් පේන ලෙස උස්සලා අන්දා සිටි බැවින් සරාගි...,NOT
1145,8645,NaN,අහසේ දෙයියෝ හුත්තෝ දෙයියෝ ඉන්නේ මෙහෙ මෙහෙ උට්ට...,HOF


In [ ]:
# validation data

df_dev1['label'].value_counts()


NOT    431
HOF    319
Name: label, dtype: int64

# Processing Tweet text and transforming to word embeddings

In [ ]:
# Pre-processing tweet

def preprocess_tweet(text):
    text = re.sub('@username', 'MENTIONED_NAME ', text)                           # Removes usernames
    text = re.sub('@AUTHOR', 'MENTIONED_NAME', text)                              # Removes usernames
    text = re.sub('@USER', 'MENTIONED_NAME', text)                                # Removes usernames
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)                  # Remove URLs
    text = re.sub(r"\d+", "", text)                                               # Removes all digits
    text = re.sub('&quot;'," ", text)                                             # Remove (&quot;)
    text = re.sub(r"\b[a-zA-Z]\b", "", str(text))                                 # Removes all single characters
    text = re.sub(r'(.)\1+', r'\1\1', text)                                       # Convert more than 2 letter repetitions to 2 letter
    text = re.sub(r"\t+", " ", text)                                              # Replaces tab spaces with single space
    text = re.sub(r"\s+", " ", text)                                              # Replaces double spaces with single space
    text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)                                # remove duplicate of consecutives words
    return text


In [ ]:
# data-train with balancing
# column name = "cleantweet" --> standard text preprocessing

shuffled_train['cleantweet'] = shuffled_train['tweet'].apply(preprocess_tweet)
shuffled_train

In [ ]:
df_test['cleantweet'] = df_test['tweet'].apply(preprocess_tweet)
df_test

In [ ]:
# data_dev for validation

df_dev1['cleantweet'] = df_dev1['tweet'].apply(preprocess_tweet)
df_dev1

In [ ]:
# data-train without balancing

df_train1['cleantweet'] = df_train1['tweet'].apply(preprocess_tweet)
df_train1

In [ ]:
# function for dealing with emoji (see the paper)

def detect_emoji(a):
    emoji_pattern = re.compile("[\U0001F600-\U0001F9FF\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF]+")

# Find all emojis in the text
    emojis = emoji_pattern.findall(a)
    emo=[]
# Remove spaces from extracted emojis
    emojis_cleaned = [emoji.replace(" ", "") for emoji in emojis]
    count_emoji = [0]
    #print(emojis_cleaned)
    new_text = a.split(' ')
    for emoji in emojis_cleaned:
        new = (re.split(r'', emoji))
        for x in new:
            #print((x))
            new_text.append(x)
    new_text1 = ' '.join(new_text)
    new_text1 = re.sub(r"\s+", " ", new_text1)
    return new_text1



In [ ]:
# column name = "ok" --> emojis are treated as special tokens

shuffled_train['ok'] = shuffled_train['cleantweet'].apply(detect_emoji)
shuffled_train


In [ ]:
# data_test (for HASOC submission)

df_test['ok'] = df_test['cleantweet'].apply(detect_emoji)
df_test

In [ ]:
df_dev1['ok'] = df_dev1['cleantweet'].apply(detect_emoji)
df_dev1

In [ ]:
# data-train for word embeddings fasttext (use all/original data train)

df_train['cleantweet'] = df_train['tweet'].apply(preprocess_tweet)
df_train['ok'] = df_train['cleantweet'].apply(detect_emoji)
df_train

In [ ]:
# PROCESSING FASTTEXT EMBEDDING
# if concerning emoji, use column ['ok']
# if use normal text processing without tokenizing emoji, use column ['cleantweet']
#
sentences = [word_tokenize(Tweet) for Tweet in tqdm(df_train['ok'])]
model = FastText(sentences, vector_size=100, window=3, min_count=2, workers=4, epochs=50)

In [ ]:
# Saving model (adjust the file name and path location)
# will be needed to alter the FastText language model

model.save("sinhala_w3c2v100.fasttext")

In [ ]:
# Load the model

model = FastText.load("sinhala_w3c2v100.fasttext")


In [ ]:
#model['නායකයා']
feat = model.wv
feat.vectors

In [ ]:
# Function for creating sentence vector

def norm_vector(sentence, model):
    vecs = [model[word] for word in word_tokenize(sentence)]
    norm_vecs = [vec / np.linalg.norm(vec) for vec in vecs if np.linalg.norm(vec) > 0]
    sent_vecs = np.mean(norm_vecs, axis=0)
    return sent_vecs

# **TRANSFORM SENTENCE INTO VECTOR**

In [ ]:
vecs_train = [norm_vector(sentence, feat) for sentence in shuffled_train.ok]
vecs_train = np.array(vecs_train)
vecs_train

array([[-5.6887668e-02, -6.8822759e-03, -7.6828472e-04, ...,
         3.2185442e-05, -5.7128023e-02, -1.5850360e-02],
       [ 3.3731952e-02, -2.4035878e-02, -1.2270814e-02, ...,
        -3.4085557e-02,  1.5071415e-02, -6.4492261e-04],
       [ 2.7488681e-02, -1.1688858e-02,  1.7720545e-02, ...,
        -3.7581824e-02, -2.8778730e-02,  3.1723484e-02],
       ...,
       [ 3.4136720e-02, -1.6505117e-02, -3.2315586e-02, ...,
        -2.9686270e-02, -8.7184804e-03, -1.4336245e-02],
       [ 2.6732562e-02, -3.2145746e-02, -3.5472650e-02, ...,
        -1.9729570e-02,  1.7658006e-02,  2.1268060e-02],
       [ 1.0084289e-02,  2.4283404e-02, -5.6376085e-03, ...,
         2.8706591e-03,  3.0506453e-02, -2.7896937e-03]], dtype=float32)

In [ ]:
vecs_dev = [norm_vector(sentence, feat) for sentence in df_dev1.ok]
vecs_dev = np.array(vecs_dev)
vecs_dev

array([[ 0.02841851,  0.00067929, -0.04173805, ..., -0.04154312,
         0.00134731, -0.0218883 ],
       [-0.01198898, -0.02833401,  0.01888742, ..., -0.03112266,
        -0.00941231, -0.01677913],
       [ 0.00467574, -0.02278483, -0.03343655, ..., -0.0515876 ,
         0.01947294, -0.01766241],
       ...,
       [ 0.0391865 ,  0.01585085,  0.00012952, ..., -0.07477746,
        -0.02469535, -0.03000988],
       [ 0.04795717, -0.00293585,  0.00259417, ..., -0.02006187,
         0.01258126,  0.00648162],
       [-0.01300191, -0.01335822, -0.039092  , ...,  0.01249252,
         0.0257376 , -0.03880717]], dtype=float32)

In [ ]:
vecs_test = [norm_vector(sentence, feat) for sentence in df_test.ok]
vecs_test = np.array(vecs_test)
vecs_test

array([[ 0.02036053,  0.0416871 , -0.03889521, ..., -0.00613656,
        -0.03478594,  0.03410538],
       [ 0.05254064,  0.0084663 , -0.06094459, ..., -0.02086362,
         0.00846146, -0.00919613],
       [-0.03986843, -0.04553909,  0.06200192, ...,  0.0010208 ,
         0.08350173, -0.00852834],
       ...,
       [ 0.00242339, -0.01687736,  0.01765568, ...,  0.02207897,
         0.03728187, -0.01602666],
       [ 0.02332435, -0.0407596 , -0.03102377, ...,  0.00578075,
         0.07404105, -0.06994012],
       [ 0.0073469 , -0.00954356, -0.01056082, ...,  0.03843581,
         0.06405949,  0.02256356]], dtype=float32)

In [ ]:
vecs_dev.shape

(750, 100)

**CLASSIFICATION PHASE**

In [ ]:

from sklearn.metrics import accuracy_score
from sklearn.svm import SVC


In [ ]:
# Train SVM and validation (Predict data-dev)

svc_HS = SVC()
svc_HS = svc_HS.fit(vecs_train, shuffled_train.label)
result_svc_noscaling = svc_HS.predict(vecs_dev)
print(accuracy_score(df_dev1.label, result_svc_noscaling)*100)

# jump to cell code evaluation scoring

78.53333333333333


In [ ]:
# Evaluation

from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, classification_report
print(classification_report(df_dev1.label, result_svc_noscaling))

#Scoring Report

              precision    recall  f1-score   support

         HOF       0.74      0.76      0.75       319
         NOT       0.82      0.80      0.81       431

    accuracy                           0.79       750
   macro avg       0.78      0.78      0.78       750
weighted avg       0.79      0.79      0.79       750



In [ ]:
# Use minmax scaler for sentence vector

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(vecs_train)

X_train_scaled = scaler.transform(vecs_train)
X_test_scaled = scaler.transform(vecs_dev)

In [ ]:
# Train SVM and validation (Predict data-dev)

svc_HS = svc_HS.fit(X_train_scaled, shuffled_train.label)
result_svc_minmax = svc_HS.predict(X_test_scaled)
print(accuracy_score(df_dev1.label, result_svc_minmax)*100)

# Accuracy

77.60000000000001


In [ ]:
# Evaluation

print(classification_report(df_dev1.label, result_svc_minmax))

#Scoring Report

              precision    recall  f1-score   support

         HOF       0.73      0.74      0.74       319
         NOT       0.81      0.80      0.80       431

    accuracy                           0.78       750
   macro avg       0.77      0.77      0.77       750
weighted avg       0.78      0.78      0.78       750



In [ ]:
# Use robust scaler for sentence vector

from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler.fit(vecs_train)

X_train_scaled = scaler.transform(vecs_train)
X_test_scaled = scaler.transform(vecs_dev)

In [ ]:
# Train SVM and validation (Predict data-dev)

svc_HS = svc_HS.fit(X_train_scaled, shuffled_train.label)
results_svc_robust = svc_HS.predict(X_test_scaled)
print(accuracy_score(df_dev1.label, results_svc_robust)*100)

# Accuracy

77.60000000000001


In [ ]:
# Evaluation

print(classification_report(df_dev1.label, results_svc_robust))

              precision    recall  f1-score   support

         HOF       0.73      0.74      0.74       319
         NOT       0.81      0.80      0.80       431

    accuracy                           0.78       750
   macro avg       0.77      0.77      0.77       750
weighted avg       0.78      0.78      0.78       750



# Parameter Tuning

In [ ]:
# grid search on parameters using default 5-fold cross validation

#default SVC param
# C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True,
#probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False,
#max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)

from sklearn.model_selection import GridSearchCV
parameters = {
     #'kernel': ['sigmoid','rbf', 'linear'],
     'C': [ 1, 10],
    }

clf = GridSearchCV(SVC(random_state=42), param_grid=parameters, verbose=1, scoring='accuracy')

clf.fit(X_train_scaled, shuffled_train.label)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


GridSearchCV(estimator=SVC(random_state=42), param_grid={'C': [1, 10]},
             scoring='accuracy', verbose=1)

In [ ]:
clf.best_params_

{'C': 10}

In [ ]:
#  Enable the line with the optimal results for assigning which vector to be used as SVC input

optimal_vectors_train = vecs_train         # no scaling
#optimal_vectors = X_train_scaled          # with scaling

optimal_vectors_val = vecs_dev             # no scaling
#optimal_vectors_val = X_test_scaled       # with scaling


best_model = SVC (kernel='rbf', C=10)
best_model.fit(optimal_vectors_train, shuffled_train.label)
print('acc for data-train\n', best_model.score(optimal_vectors_train, shuffled_train.label))

final_result = best_model.predict(optimal_vectors_val)
print('\nevaluation reports for data-dev (validation)\n')
print(classification_report(df_dev1.label, final_result))

acc for data-train
 0.9202414590290264

evaluation reports for data-dev (validation)

              precision    recall  f1-score   support

         HOF       0.70      0.75      0.72       319
         NOT       0.80      0.76      0.78       431

    accuracy                           0.75       750
   macro avg       0.75      0.75      0.75       750
weighted avg       0.76      0.75      0.76       750



In [ ]:
# SUBMISSION

In [ ]:
# select vectors depend on which scaling is the best

X_test_real = vecs_test  #no scaling
#X_test_real = scaler.transform(vecs_test)

result = svc_HS.predict(X_test_real)
result

array(['HOF', 'HOF', 'HOF', ..., 'HOF', 'HOF', 'HOF'], dtype=object)